Notebook responsável pela criação da tabela dimensão relacionada aos tipos de viagem (trip_type ).

Informações retiradas das documentações em:

https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf
https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf

In [0]:
import pyspark.sql.functions as F
import yaml

In [0]:
# método responsável por incluir os metadados nas colunas da tabela. 
# ele lê um arquivo yaml com a configuração das colunas e atualiza os metadados
def create_update_column_metadata(table_name, file_path):
    try:
        with open(file_path, "r") as file:
            column_comments = yaml.safe_load(file)["columns"]

            for column, comment in column_comments.items():
                spark.sql(f"ALTER TABLE {table_name} ALTER COLUMN {column} COMMENT '{comment}'")
    except Exception as e:
        print(f"Error during update column metadata: {e}")

In [0]:
trip_type_data = [
    (1, "Street-hail"),
    (2, "Dispatch")
]

table_name = "tb_dim_trip_type"
silver_table_name = f"ifood_case.silver.{table_name}"

df_silver = spark.createDataFrame(trip_type_data, ["id_trip_type", "ds_trip_type_name"])

In [0]:
df_silver.write.format("delta").mode("overwrite").saveAsTable(f"{silver_table_name}")

In [0]:
create_update_column_metadata(silver_table_name, f"./metadata/{table_name}.yaml")